In [1]:
from sklearn.externals import joblib
import pandas as pd

In [3]:
import hhgen.optimize_utils as optimization

In [2]:
X_train=pd.read_hdf("X_train_10.hd5")
y_train=pd.read_hdf("y_train_10.hd5")
w_train=pd.read_hdf("w_train_10.hd5")
clf = joblib.load('clf_10.joblib')

In [4]:
param_grid = {"n_estimators": [300,500,800],
              "max_depth": [3,5,7,10],                                                                                                                                                                                                
              "reg_lambda":[0.1, 0.5, 1, 2, 10],
              "learning_rate": [0.1, 0.01, 0.5],
              "subsample" : [0.6, 0.8, 1]}

optimization.optimize_parameters_randomizedCV(clf,X_train,y_train,param_grid,cvOpt=5,weights=w_train)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/swshare/anaconda/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/mnt/t3nfs01/data01/swshare/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

......................